# Data Analysis

Now that we've got clean data, let's start with some basic financial analysis.

First, let's load our CSV file into a DataFrame, covert our dates, set the index, and check for duplicated rows or missing values.

In [1]:
import pandas as pd

df = pd.read_csv("TSLA_10_clean.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date").sort_index().drop_duplicates()

print("Duplicates:", df.duplicated().sum())
print("Missing:", df.isnull().sum().sum())

Duplicates: 0
Missing: 0


## Returns

Returns refer to the gain or loss made on an initial investment, often expressed as a percentage. We can use the generic **percentage change** formula here:

$$ (price_{end} - price_{start}) / price_{start} $$

We can apply this to close prices to calculate the simple daily return:

$$ (close price_{today} - close price_{yesterday}) / close price_{yesterday} $$

In [2]:
jan31_closing = df.loc["2024-01-31", "Close"]
jan30_closing = df.loc["2024-01-30", "Close"]

jan31_return = (jan31_closing - jan30_closing) / jan30_closing
print(f"Return on 31 Jan was {jan31_return:.2%}")

Return on 31 Jan was -2.24%


This simple daily return expresses a loss in value of 2.24% from one day to the next. Notice we leave our return in decimal form, but when we output it we use `f-strings` and `:.2%` to display it as a percentage.

If we wanted to use the above approach to calculate daily returns for each day in our data set, it would take a long time. Let's see how we can use pandas `pct_change()` to make this sort of work easy, by applying our percentage change formula one column at a time.

In [3]:
# Create a new column and populate it with daily returns
df['DailyReturn'] = df['Close'].pct_change()
df

,Close,High,Low,Open,Volume,DailyReturn
Date,,,,,,
2015-01-02,14.620667,14.883333,14.217333,14.858000,71466000,NaN
2015-01-05,14.006000,14.433333,13.810667,14.303333,80527500,-0.042041
2015-01-06,14.085333,14.280000,13.614000,14.004000,93928500,0.005664
2015-01-07,14.063333,14.318667,13.985333,14.223333,44526000,-0.001562
2015-01-08,14.041333,14.253333,14.000667,14.187333,51637500,-0.001564
...,...,...,...,...,...,...
2024-12-23,430.600006,434.510010,415.410004,28.527332,72698100,0.022657
2024-12-24,462.279999,462.779999,435.140015,435.899994,59551800,0.073572
2024-12-26,454.130005,465.329987,451.019989,465.160004,76366400,-0.017630


Notice how the first row in our data has a missing value **NaN** in the new daily return column. This is because our data doesn't have a close price for the day before it!

What to do with this missing value depends on what further analysis we want to do. If we want to carry out simple descriptive statistics like compute the mean, max, or standard deviation,  we can leave our missing value as NaN, because pandas will by default ignore NaNs when calculating these.

In [4]:
print(f"Average daily return: {df['DailyReturn'].mean():.2%}")

print(f"Highest daily return: {df['DailyReturn'].max():.2%}")

print(f"Daily Volatility: {df['DailyReturn'].std():.2%}")

Average daily return: 2.19%
Highest daily return: 1658.69%
Daily Volatility: 45.45%


For more complex analyses though, we may want to drop or fill this value. Let's calculate cumulative returns for the period. Instead of comparing a given day with the day before it, cumulative returns compare a given day with the first day of the period, to indicate how our stock has performed since our initial investment.

We generally fill missing daily returns with a 0, which indicates no change with the day before.

Because we're doing cumulative multiplication, we'll add 1 to the closing price to get a growth factor, so we can compound the return over time. After calculating the cumulative product, we subtract 1 to get back to a return.

In [5]:
df["DailyReturn"] = df["DailyReturn"].fillna(0)

df["CumulativeReturn"] = (1 + df["DailyReturn"]).cumprod() - 1

df

,Close,High,Low,Open,Volume,DailyReturn,CumulativeReturn
Date,,,,,,,
2015-01-02,14.620667,14.883333,14.217333,14.858000,71466000,0.000000,0.000000
2015-01-05,14.006000,14.433333,13.810667,14.303333,80527500,-0.042041,-0.042041
2015-01-06,14.085333,14.280000,13.614000,14.004000,93928500,0.005664,-0.036615
2015-01-07,14.063333,14.318667,13.985333,14.223333,44526000,-0.001562,-0.038120
2015-01-08,14.041333,14.253333,14.000667,14.187333,51637500,-0.001564,-0.039624
...,...,...,...,...,...,...,...
2024-12-23,430.600006,434.510010,415.410004,28.527332,72698100,0.022657,28.451460
2024-12-24,462.279999,462.779999,435.140015,435.899994,59551800,0.073572,30.618255
2024-12-26,454.130005,465.329987,451.019989,465.160004,76366400,-0.017630,30.060826


### Exercise: Bull or Bear?

Calculate the daily change in trading volume over the data frame. Then calculate the average change in trading volume over the period. Take the same approach we used for calculating daily returns, but considering `Volume` instead of `Close` price.

Then determine the trend in TSLA's stock for **Q1 2024**:

- Rising volume and increasing price might indicate a **bullish** trend (where the uptrend is backed by strong demand and could continue).

- Rising volume and decreasing price might indicate a **bearish** trend (where the downtrend is backed by strong selling pressure and could continue).

- Falling volume on price increase or decrease often indicate a trend is losing strength. It might suggest that momentum is waning and a price reversal is coming.

In [13]:
## YOUR CODE GOES HERE

import pandas as pd

# 🔹 Step 1: 读取并预处理数据
df = pd.read_csv('/workspaces/python-for-finance-Joey-9799/Completed/TSLA_10_clean.csv')
df.columns = df.columns.str.strip()
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# 🔹 Step 2: 计算每日交易量变化
df['Volume_Change'] = df['Volume'].diff()

# 🔹 Step 3: 计算日均交易量变化
average_volume_change = df['Volume_Change'].mean()
print(f"Average daily change in trading volume: {average_volume_change:,.0f}")

# 🔹 Step 4: 筛选 Q1 2024 数据
df = df.sort_index()

q1_2024 = df.loc['2024-01-01':'2024-03-31']

# 🔹 Step 5: 分析 Q1 2024 的价格与成交量变化
price_start = q1_2024['Close'].iloc[0]
price_end = q1_2024['Close'].iloc[-1]
price_change_pct = (price_end - price_start) / price_start * 100

volume_start = q1_2024['Volume'].iloc[0]
volume_end = q1_2024['Volume'].iloc[-1]
volume_change_pct = (volume_end - volume_start) / volume_start * 100

print(f"\n  Price change in Q1 2024: {price_change_pct:.2f}%")
print(f"Volume change in Q1 2024: {volume_change_pct:.2f}%")

# 🔹 Step 6: 判断趋势类型
if price_change_pct > 0 and volume_change_pct > 0:
    print(" Bullish: Price and volume are both rising — strong uptrend with buying demand.")
elif price_change_pct < 0 and volume_change_pct > 0:
    print(" Bearish: Price is falling but volume is rising — potential strong selling pressure.")
elif volume_change_pct < 0:
    print(" Weakening trend: Volume is declining — up/downtrend may be losing momentum.")
else:
    print(" Mixed or neutral signals — more analysis may be needed.")


Average daily change in trading volume: -44,024

  Price change in Q1 2024: -29.24%
Volume change in Q1 2024: -25.80%
 Weakening trend: Volume is declining — up/downtrend may be losing momentum.


## Surges

Surges in price or trading volume can be helpful indicators for traders. One way to define a surge is as an increase on the day before by an amount higher than some defined threshold. That threshold is often defined as some number of standard deviations above the mean. Let's look at price surges of five standard deviations above the mean.

In [ ]:
# Find the mean return
mean_return = df["DailyReturn"].mean()

# Define a threshold as two standard deviations above the mean
return_threshold = mean_return + (df["DailyReturn"].std() * 5)

# Define a condition
condition = df["DailyReturn"] > return_threshold

# Subset the dataframe where daily returns are higher than the threshold
df[condition]

## Moving Averages

Moving averages are a different kind of indicator, one that smooths out small variations in trading data to give a better picture of the overall trend.

A Simple Moving Average (SMA) is one which averages out a price over a specific period. The average is "moving" because when a new day is considered in the period, the oldest date is discarded.

Moving averages can be *fast*, when they cover a short period, or *slow* when they consider a longer period. The longer the period, the more those small variations are smoothed out.

In [ ]:
# Calculate a fast, 20-Day Moving Average
df['FastMA'] = df['Close'].rolling(window=20).mean()

# Calculate a slow, 200-Day Moving Average
df['SlowMA'] = df['Close'].rolling(window=200).mean()

## Volatility

Volatility looks at the degree of variance in a stock, and can be helpful for determining risk. Periods of high standard deviation indicate higher volatility and may suggest a riskier investment.


In [ ]:
# Calculate 20-Day Standard Deviation
df['RollingVolatility'] = df['DailyReturn'].rolling(window=20).std()
df